In [1]:
!pip install  tabulate

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents.agent_types import AgentType
from google.colab import userdata


In [5]:
csv_file_path = '/content/sales_report.csv'

In [6]:
# Step 1: Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = userdata.get("OPENROUTER_OPENAI") # Replace with your actual OpenRouter API key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",  # or try "meta-llama/llama-3-8b-instruct", etc.
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
    verbose=True
)

/tmp/ipython-input-1652786475.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
print("\nCreating CSV Agent...")
agent_executor = create_csv_agent(
    llm,
    csv_file_path,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True, # Allow the agent to execute python code
    handle_parsing_errors=True # Handle cases where the LLM output is not perfectly formatted

)
print("Agent is ready.")

In [7]:
AGENT_PREFIX = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you.
You have access to a single tool: `python_repl_ast`.
Use this to process the dataframe and return the final answer.

Your response MUST be in the following format:

Question: the user's question
Thought: I need to use pandas to find the answer. I will write the code to do that.
Action: python_repl_ast
Action Input: a single, valid line of pandas code to run

Example:
Question: how many rows are there?
Thought: I need to count the rows in the dataframe.
Action: python_repl_ast
Action Input: df.shape[0]
"""

print("\nCreating CSV Agent with a guiding prefix...")
agent_executor = create_csv_agent(
    llm,
    csv_file_path,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    prefix=AGENT_PREFIX
)
print("Agent is ready.")


Creating CSV Agent with a guiding prefix...
Agent is ready.


/usr/local/lib/python3.12/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [8]:
query1 = "What was the total sale amount for the North region?"
print(f"\nQuestion: {query1}")
response1 = agent_executor.invoke(query1)
print("Answer:", response1['output'])



Question: What was the total sale amount for the North region?


> Entering new AgentExecutor chain...
 <s>[ASSISTANT] Question: What was the total sale amount for the North region?
Thought: I need to filter the dataframe for rows where the Region is 'North' and then sum the 'Sale Amount' column.
Action: python_repl_ast
Action Input: df[df['Region'] == 'North']['Sale Amount'].sum()17000 <s>[ASSISTANT]
I now know the final answer
Final Answer: The total sale amount for the North region was 17000.

> Finished chain.
Answer: The total sale amount for the North region was 17000.


In [ ]:
query1 = "how many laptops sold in south region"
print(f"\nQuestion: {query1}")
response1 = agent_executor.invoke(query1)
print("Answer:", response1['output'])